# 🪐 Transmission Spectroscopy of Exoplanet Atmospheres



## What is Transmission Spectroscopy?

When an exoplanet transits (passes in front of) its host star, a small fraction of the starlight filters through the planet’s atmosphere.

Different wavelengths are absorbed by different molecules — causing the **apparent transit depth** to vary with wavelength.

This variation encodes information about the atmosphere’s:
- Composition (via molecular absorption features)
- Scale height (via temperature and mean molecular weight)
- Clouds/hazes (which can mute spectral features)


In [9]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider
from scipy.constants import k, G, m_p

## 📈 The Physical (Toy) Model

For transmission spectroscopy, the **effective radius** of the planet at a given wavelength $R_p(\lambda)$ depends on how high the atmosphere is transparent or opaque.

The **transit depth** is:

$$
\Delta_\lambda = \left(\frac{R_p(\lambda)}{R_*}\right)^2
$$

where $R_*$ is the stellar radius.

We’ll approximate the atmospheric contribution as:

$$
R_p(\lambda) = R_{p,0} + N_\text{scale}(\lambda) \, H
$$

where:
- $R_{p,0}$ = reference planetary radius (without atmosphere)
- $H = \frac{kT}{\mu m_p g}$ = atmospheric scale height
- $N_\text{scale}(\lambda)$ = the number of scale heights which are opaque at a given wavelength. E.g. if the atmosphere is very opaque at a certain wavelength, the planet will look larger and $N_\text{scale}$ will be larger.

We’ll also add a **cloud deck** at a certain level in the atmosphere — this blocks our view of anything below the cloud deck, i.e. it will set a baseline value of $N_\text{scale}(\lambda)$.


In [10]:
# --- Planetary and stellar parameters for GJ 1214 b ---
R_star = 0.216 * 6.96e8       # stellar radius (m)
M_p = 6.26 * 5.972e24         # planet mass (kg)
R_p0 = 2.85 * 6.371e6         # reference radius (m)
T_eq = 600                    # equilibrium temperature (K)
g = G * M_p / R_p0**2         # surface gravity (m/s^2)

print(f"Surface gravity of GJ 1214 b: {g:.2f} m/s^2")

# --- Toy model of how N_scale varied with wavelength ---
def N_scale(wavelength_microns):
    """
    Generic opacity function for demonstration purposes.
    Simulates molecular absorption features.
    """
    lam = wavelength_microns
    Nsc = 1.0 + 3.0*np.exp(-0.5*((lam - 1.4)/0.1)**2) + 5.0*np.exp(-0.5*((lam - 3.0)/0.15)**2)
    return Nsc

def N_cloud(wavelength_microns, cloud_scale_heights):
    """
    Simple gray opacity that increases at long wavelengths.
    """
    return cloud_scale_heights * np.ones_like(wavelength_microns)


Surface gravity of GJ 1214 b: 7.57 m/s^2


## 💻 Computing the Transmission Spectrum

We’ll now compute the **effective radius** $R_p(\lambda)$ and the **transit depth** $\delta_\lambda$.

Key dependencies:
- **Mean molecular weight (μ):** heavier gases → smaller scale height → weaker features.
- **Cloud-top pressure:** higher clouds block light from deeper layers, muting spectral features.

We’ll let you explore these interactively below.


In [11]:
def transmission_spectrum(mu=2.3, cloud_scale_heights=0.0):
    """
    Compute a toy transmission spectrum for given mean molecular weight and cloud top.
    """
    # Wavelength grid (microns)
    lam = np.linspace(1.0, 5.0, 300)

    # Compute scale height
    H = k * T_eq / (mu * m_p * g)

    # Opaque scale heights
    N_tot = np.maximum(N_scale(lam), N_cloud(lam, cloud_scale_heights))

    # Relative radius variation (log of opacity)
    Rp_lam = R_p0 + (N_tot * H)

    # Convert to transit depth
    delta = (Rp_lam / R_star)**2 * 1e6  # in ppm

    # Plot
    plt.figure(figsize=(8,5))
    plt.plot(lam, delta, color='blue')
    plt.ylim([14500,18000])
    plt.xlabel("Wavelength (μm)")
    plt.ylabel("Transit depth (ppm)")
    plt.title(f"GJ 1214 b Transmission Spectrum (μ={mu:.1f}, P_cloud={cloud_scale_heights:.2f} bar)")
    plt.grid(alpha=0.3)
    plt.show()


In [ ]:
interact(
    transmission_spectrum,
    mu=FloatSlider(min=2.0, max=30.0, step=1.0, value=2.3, description='Mean μ'),
    cloud_scale_heights=FloatSlider(min=0.0, max=5.0, step=0.01, value=0.1, description='N_cloud')
);


interactive(children=(FloatSlider(value=2.3, description='Mean μ', max=30.0, min=2.0, step=1.0), FloatSlider(v…

## 🧩 Interpretation

Try adjusting the sliders above!

### Mean Molecular Weight (μ)
- **Low μ (~2):** hydrogen-rich atmosphere → large scale height → strong spectral features.  
- **High μ (~20–30):** heavier molecules → smaller scale height → flat, muted spectrum.

### Cloud-Top Pressure
- **High clouds (low pressure):** flatten the spectrum → less spectral contrast.  
- **Deep clouds (high pressure):** allow more of the molecular features to appear.

